In [1]:
import numpy as np
import pickle
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

plt.style.use("ggplot")

import json

In [2]:
df = pd.read_excel('../data/Data_2020_P06/all_billings_inputs_08_20.xlsx',
                  sheet_name = 'base_billings')

In [3]:
print('df length: ', len(df))

df length:  294044


In [4]:
df.rename(
        index=str,
        columns={
            "Contrct Duration in Months": "duration",
            "Document Currency": "curr",
            "Enterprise BU Desc": "BU",
            "Invoice Fiscal Year Period Desc": "period",
            "POB Type": "POB_type",
            "Product Config Type": "config",
            "Rev Rec Category": "rev_req_type",
            "Rule For Bill Date": "rebill_rule",
            "Sales Document Type": "sales_doc",
            "Sales Type": "sales_type",
            "Subscription Term": "sub_term",
            "Completed Sales ( DC )": "DC_amount",
            "Completed Sales": "US_amount",
        },
        inplace=True,
    )


In [5]:
df.head(3)

,duration,curr,BU,period,POB_type,config,rev_req_type,rebill_rule,sales_doc,sales_type,sub_term,DC_amount,US_amount
0,0,AUD,Experience Cloud,2017-01,NaN,OCONS,NaN,NaN,ZCC,PRO-SVC-INV,0,375.0,278.01
1,0,AUD,Experience Cloud,2017-01,NaN,ONORE,NaN,NaN,ZCC,RECOGNIZED,0,0.0,45.75
2,0,AUD,Experience Cloud,2017-01,NaN,ONORE,D,YA,ZCC,DEFERRED,0,-684738.0,-512177.81


In [6]:
vc = df["curr"].value_counts()
keep_these = vc.values > 20
keep_curr = vc[keep_these]
list_keepers = keep_curr.index
remove_these = vc[vc.values <= 20].index
model_dict = {"curr_removed": list(vc[remove_these].index)}
delete_curr = list(remove_these)

if "TRY" not in model_dict["curr_removed"]:
    model_dict["curr_removed"].append("TRY")
    delete_curr.append("TRY")
    list_keepers = list_keepers.drop("TRY")

df = df[df["curr"].isin(list_keepers)]

In [7]:
print(len(df))
df = df[df["DC_amount"] != 0]
print(len(df))

293832
286744


In [8]:
model_dict

{'curr_removed': ['BHD',
  'JOD',
  'EGP',
  'OMR',
  'LBP',
  'BMD',
  'AED',
  'MXP',
  'TRY']}

In [9]:
df['POB_type'].value_counts(dropna=False)

NaN      192219
RR        73256
IR-NA      7424
IR         4819
BNDL       3604
CR         2956
RR-NA      2308
CR-NA       102
LFB          56
Name: POB_type, dtype: int64

## Grouping by POB Type

The new 606 has everything grouped by sales type. There are blanks
We need to separate into groups based on the POB type


In [10]:
list_IR = ['IR', 'IR-NA', 'LFB']
list_service = ['CR', 'CR-NA']
list_deferred = ['RR', 'RR-NA']
list_hybrid = ['BNDL']

list_all = list_IR + list_service + list_deferred + list_hybrid
print(list_all)

['IR', 'IR-NA', 'LFB', 'CR', 'CR-NA', 'RR', 'RR-NA', 'BNDL']


In [11]:
rec = df[df["POB_type"].isin(list_IR)].copy()
svc = df[df["POB_type"].isin(list_service)].copy()
dfr = df[df["POB_type"].isin(list_deferred)].copy()
hyb = df[df["POB_type"].isin(list_hybrid)].copy()
blank = df[~df["POB_type"].isin(list_all)].copy()

In [12]:
print('rec', len(rec))
print('svc', len(svc))
print('dfr', len(dfr))
print('hyb', len(hyb))
print('blank', len(blank))
print('ALL', len(df))

rec 12299
svc 3058
dfr 75564
hyb 3604
blank 192219
ALL 286744


In [ ]:
hyb.head(10)

In [ ]:
def split_hybrid_dataframe(hyb, pct_IR):
    df_IR = hyb.copy()
    df_deferred = hyb.copy()
    
    df_IR['DC_amount'] = df_IR['DC_amount']* pct_IR
    df_IR['US_amount'] = df_IR['US_amount']* pct_IR  
    
    df_deferred['DC_amount'] = df_deferred['DC_amount']* (1 - pct_IR)
    df_deferred['US_amount'] = df_deferred['US_amount']* (1 - pct_IR)
    
    return df_IR, df_deferred

In [ ]:
pct_IR = .17
df_IR, df_defered = split_hybrid_dataframe(hyb, pct_IR)


In [ ]:
print(hyb['US_amount'].sum())
print(df_IR['US_amount'].sum())
print(df_defered['US_amount'].sum())

In [ ]:
df_IR['US_amount'].sum()+df_defered['US_amount'].sum()

In [ ]:
len(rec)+len(svc)+ len(dfr)+ len(blank)+len(hyb)

In [ ]:
gb_rec = rec.groupby(["curr", "BU", "period"], as_index=False).sum()

In [ ]:
gb_rec.drop(labels=["duration", "sub_term"] , axis=1, inplace=True)

In [ ]:
gb_rec

In [ ]:
gb_svc = svc.groupby(["curr", "BU", "period"], as_index=False).sum()

In [ ]:
gb_svc.drop(labels=["sub_term", "duration"] , axis=1, inplace=True)

gb_svc

In [ ]:
gb_svc.head(4)

# Deferred billings
## Type B Service based/

In [ ]:
dfr_b = dfr[dfr["rev_req_type"] == "B"].copy()

In [ ]:
print(len(dfr))
print(len(dfr_b))

In [ ]:
dfr.head(10)

In [ ]:
dfr.rev_req_type.value_counts(dropna=False)

## Type A Deferred billings

### First sort by config type

### Then use sub term

In [ ]:
dfr_a = dfr[dfr["rev_req_type"] == "A"].copy()
print("length of dfr_a", len(dfr_a))
dfr_a.drop(labels='duration', axis=1, inplace=True)
gb_a = dfr_a.groupby(["curr", "BU", "period", "config", 'sub_term'], as_index=False).sum()


In [ ]:
len(gb_a)

In [ ]:
gb_a.head(4)


In [ ]:
gb_a.config.value_counts(dropna=False)

In [ ]:
gb_a.sub_term.value_counts(dropna=False)

In [ ]:
config_type_keepers = ['MTHLY', '1Y', '2Y', '3Y']


In [ ]:
gb_a_keepers = gb_a[gb_a["config"].isin(config_type_keepers)].copy()
a_bad_config = gb_a[~gb_a["config"].isin(config_type_keepers)].copy()

In [ ]:
print('len gb_a', len(gb_a))
print('gb_a_keepers', len(gb_a_keepers))
print('len a_blank_config', len(a_bad_config))


In [ ]:
gb_a_keepers['config'].value_counts(dropna=False)

In [ ]:
gb_a_keepers['sub_term'].value_counts(dropna=False)

In [ ]:
gb_a_keepers.head(10)

In [ ]:
print('Total USD Equivalent Billings of Type A with bad configs', 
      a_bad_config.US_amount.sum())

## For the type A billings that we can use, we need to split them apart by sub_term and config
If subterm == 0, then we use the config period. (Assuming these are PUP?)

Otherwise, 12 = annual, 24 = two years, 36 = 3 years

Old code has these as gb_a_1M, gb_a_1Y, gb_a_2Y, gb_a_3Y

Here we will pull out the types from the gb_a_keepers dataframe into a new dataframe and then delete these old records from the gb_a_keepers dataframe

In [ ]:
print('start length', len(gb_a_keepers))
df_1M = gb_a_keepers[(gb_a_keepers['config']=='MTHLY') |
                       (gb_a_keepers['sub_term']==1)].copy()

index_1M = df_1M.index


gb_a_keepers.drop(index_1M, inplace=True)

print(df_1M.head(10))
print('len df_1M', len(df_1M))
print(len(gb_a_keepers))


In [ ]:
print(len(gb_a_keepers))

In [ ]:
#df_1M['sub_term'].value_counts(dropna=False)
df_1M['config'].value_counts(dropna=False)

In [ ]:
gb_a_keepers.head(10)

##### Dealing with the 1 year rebillings

In [ ]:
print('start length', len(gb_a_keepers))
df_1Y = gb_a_keepers[(gb_a_keepers['sub_term']==12) | 
                       ((gb_a_keepers['sub_term']==0) &
                        (gb_a_keepers['config']=='1Y'))].copy()

index_1Y = df_1Y.index


gb_a_keepers.drop(index_1Y, inplace=True)

print(df_1Y.head(10))
print('len df_1Y', len(df_1Y))
print(len(gb_a_keepers))

In [ ]:
df_2Y = gb_a_keepers[gb_a_keepers['config']=='2Y'].copy()
df_3Y = gb_a_keepers[gb_a_keepers['config']=='3Y'].copy()

print('length gb_a_keepers', len(gb_a_keepers))
print('length df_2Y', len(df_2Y))
print('df_3Y', len(df_3Y))

In [ ]:
test2 = gb_a_keepers[gb_a_keepers['config']=='3Y']
test2.sub_term.value_counts()

# Type D Billings



In [ ]:
dfr_d = dfr[dfr["rev_req_type"] == "D"].copy()
dfr_d.head(10)

In [ ]:
gb_d = dfr_d.groupby(["curr", "BU", "period", "rebill_rule", "sales_doc"], as_index=False).sum()
gb_d.head(10)

In [ ]:
gb_d.drop(labels=["sub_term", "duration"], axis=1, inplace=True)
gb_d.head(10)

In [ ]:
gb_d["rebill_rule"].value_counts(dropna=False)

In [ ]:
list_monthly = ['Y1', 'Y2', 'Y3', 'YM']
list_qtrly = ['YQ', 'YY', 'YT']
list_semi_ann = ['YH']
list_ann = ['YA', 'YC', 'YX']
list_2yrs = ['Y4']
list_3yrs = ['Y7']
list_all_rebills = list_monthly + list_qtrly + list_semi_ann + list_ann + list_2yrs + list_3yrs
print(list_all_rebills)

In [ ]:
gb_d_mthly = gb_d[gb_d["rebill_rule"].isin(list_monthly)].copy()
gb_d_mthly.drop(labels="rebill_rule", axis=1, inplace=True)
gb_d_mthly = gb_d_mthly.groupby(["curr", "BU", "period"]).sum()
gb_d_mthly.reset_index(inplace=True)

gb_d_qtrly = gb_d[gb_d["rebill_rule"].isin(list_qtrly)].copy()
gb_d_qtrly.drop(labels="rebill_rule", axis=1, inplace=True)
gb_d_qtrly = gb_d_qtrly.groupby(["curr", "BU", "period"]).sum()
gb_d_qtrly.reset_index(inplace=True)

gb_d_semi_ann = gb_d[gb_d["rebill_rule"].isin(list_semi_ann)]
gb_d_semi_ann.drop(labels="rebill_rule", axis=1, inplace=True)
gb_d_semi_ann = gb_d_semi_ann.groupby(["curr", "BU", "period"]).sum()
gb_d_semi_ann.reset_index(inplace=True)

gb_d_annual = gb_d[gb_d["rebill_rule"].isin(list_ann)].copy()
gb_d_annual.drop(labels="rebill_rule", axis=1, inplace=True)
gb_d_annual = gb_d_annual.groupby(["curr", "BU", "period"]).sum()
gb_d_annual.reset_index(inplace=True)

gb_d_two_yrs = gb_d[gb_d["rebill_rule"].isin(list_2yrs)].copy()
gb_d_two_yrs.drop(labels="rebill_rule", axis=1, inplace=True)
gb_d_two_yrs = gb_d_two_yrs.groupby(["curr", "BU", "period"]).sum()
gb_d_two_yrs.reset_index(inplace=True)

gb_d_three_yrs = gb_d[gb_d["rebill_rule"].isin(list_3yrs)]
gb_d_three_yrs.drop(labels="rebill_rule", axis=1, inplace=True)
gb_d_three_yrs = gb_d_three_yrs.groupby(["curr", "BU", "period"]).sum()
gb_d_three_yrs.reset_index(inplace=True)

print("Length of monthly", len(gb_d_mthly))
print("Length of quarterly", len(gb_d_qtrly))
print("Length of semi ann", len(gb_d_semi_ann))
print("Length of annual", len(gb_d_annual))
print("Length of two years", len(gb_d_two_yrs))
print("Length of three years", len(gb_d_three_yrs))


In [ ]:
list_df = [
    gb_rec,
    gb_svc,
    gb_b,
    gb_a_1M,
    gb_a_1Y,
    gb_a_2Y,
    gb_a_3Y,
    gb_d_mthly,
    gb_d_qtrly,
    gb_d_semi_ann,
    gb_d_annual,
    gb_d_two_yrs,
    gb_d_three_yrs,
]

list_columns = [
    "recognized",
    "service",
    "deferred_B",
    "deferred_1M_a",
    "deferred_1Y_a",
    "deferred_2Y_a",
    "deferred_3Y_a",
    "deferred_1M_d",
    "deferred_3M_d",
    "deferred_6M_d",
    "deferred_1Y_d",
    "deferred_2Y_d",
    "deferred_3Y_d",
]


In [13]:
# Blank
blank.head(10)

,duration,curr,BU,period,POB_type,config,rev_req_type,rebill_rule,sales_doc,sales_type,sub_term,DC_amount,US_amount
0,0,AUD,Experience Cloud,2017-01,NaN,OCONS,NaN,NaN,ZCC,PRO-SVC-INV,0,375.00,278.01
2,0,AUD,Experience Cloud,2017-01,NaN,ONORE,D,YA,ZCC,DEFERRED,0,-684738.00,-512177.81
3,0,AUD,Experience Cloud,2017-01,NaN,ONORE,D,YQ,ZCC,DEFERRED,0,16196.47,11935.54
4,0,AUD,Experience Cloud,2017-01,NaN,OUNIV,B,NaN,ZCC,DEFERRED,0,-38000.00,-27070.00
5,0,CAD,Experience Cloud,2017-01,NaN,ONORE,D,YQ,ZCC,DEFERRED,0,25171.20,18816.73
6,0,EUR,Experience Cloud,2017-01,NaN,OCONS,NaN,NaN,ZCC,PRO-SVC-INV,0,525.00,552.92
7,0,EUR,Experience Cloud,2017-01,NaN,OCONS,B,NaN,ZCC,DEFERRED,0,-3790.50,-4464.85
8,0,EUR,Experience Cloud,2017-01,NaN,OCONS,D,YA,ZCC,DEFERRED,0,17256.00,18352.45
9,0,EUR,Experience Cloud,2017-01,NaN,OCONS,D,YC,ZCC,DEFERRED,0,-17256.00,-17982.82
10,0,EUR,Experience Cloud,2017-01,NaN,ONORE,D,Y1,ZCC,DEFERRED,0,-13868.36,-15248.40


In [14]:
blank.tail(10)

,duration,curr,BU,period,POB_type,config,rev_req_type,rebill_rule,sales_doc,sales_type,sub_term,DC_amount,US_amount
294034,96,USD,Creative,2020-06,NaN,1Y,D,Y3,ZCSB,DEFERRED,1,930.83,930.83
294035,97,DKK,Creative,2020-06,NaN,1Y,D,Y3,ZCSB,DEFERRED,1,370.00,54.02
294036,97,EUR,Creative,2020-06,NaN,1Y,D,Y3,ZCSB,DEFERRED,1,249.95,271.09
294037,97,JPY,Creative,2020-06,NaN,1Y,D,Y3,ZCSB,DEFERRED,1,34080.00,317.46
294038,97,USD,Creative,2020-06,NaN,1Y,D,Y3,ZCSB,DEFERRED,1,1094.79,1094.79
294039,98,JPY,Creative,2020-06,NaN,1Y,D,Y3,ZCSB,DEFERRED,1,5680.00,52.77
294040,98,USD,Creative,2020-06,NaN,1Y,D,Y3,ZCSB,DEFERRED,1,208.96,208.96
294041,99,AUD,Creative,2020-06,NaN,1Y,D,Y3,ZCSB,DEFERRED,1,139.98,90.40
294042,99,JPY,Creative,2020-06,NaN,1Y,D,Y3,ZCSB,DEFERRED,1,-5680.00,-52.26
294043,99,USD,Creative,2020-06,NaN,1Y,D,Y3,ZCSB,DEFERRED,1,211.96,211.96


In [15]:
sales_doc_type = {'immediate_revenue': ['G2', 'KR', 'OR', 'ZCCR',
                                        'ZCDR', 'ZCOR', 'ZCRE', 'ZLO3',
                                       'ZLO1', 'ZLO2', 'ZGO2', 'ZOL2', 'ZOTM' ],
                  'service':['ZATS'],
                  'deferred_D': ['ZCSB'],
                  'rev_rec_class': ['ZCC', 'ZCQ', 'ZLCR', 'ZLDR', 'ZLG2', 'ZLO5',
                                    'ZRG2', 'ZRL2']
                 }
# OH SHIT ZCC, ZCQ, ZLCR, ZLDR, ZLG2, ZLO5, ZL05, ZRG2, ZRL2
# ZCC are mostly type D. Some are type A with blank config, some are type A with OCONS or other configs
#  ZCC A have no config - immediate
# some are type B - which we should switch immediately to service
# most are type D
# Many are blank move to immediate revenue

# ZCQ - some A, moxst are D, some are blank


In [16]:
sales_doc_type

{'immediate_revenue': ['G2',
  'KR',
  'OR',
  'ZCCR',
  'ZCDR',
  'ZCOR',
  'ZCRE',
  'ZLO3',
  'ZLO1',
  'ZLO2',
  'ZGO2',
  'ZOL2',
  'ZOTM'],
 'service': ['ZATS'],
 'deferred_D': ['ZCSB'],
 'rev_rec_class': ['ZCC',
  'ZCQ',
  'ZLCR',
  'ZLDR',
  'ZLG2',
  'ZLO5',
  'ZRG2',
  'ZRL2']}

#### TODO:
 - create function that moves through these by rev_rec_class and classifies them based on if they are A, B, D or F
 - classify them as in A and D
 - return  dataframes
     -IR
     -service
     -deferred
 - merge these with the other dataframes
 - Move on to type A

In [13]:
blank['sales_doc'].value_counts(fillna=False)

TypeError: value_counts() got an unexpected keyword argument 'fillna'